In [1]:
import string
import requests
import requests_cache
from requests_cache import CachedSession
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from requests_html import HTMLSession

In [2]:
#Tao cache
def check_cached(response: requests.Response) ->bool :
    if(response.text.find('"incomplete_results":true')==-1):
        return True
    else:
        return False
    
requests_cache.install_cache(expire_after = -1, filter_fn = check_cached)

In [38]:
index = []
country = []
total_cases = []
total_deaths = []
total_recovered = []
active_cases = []
serious_critical = []
tot_cases = []
deaths = []
total_tests = []
tests = []
population = []
region = []
case_every = []
death_every = []
test_every = []

In [46]:
url = "https://www.worldometers.info/coronavirus/#nav-yesterday"

In [47]:
driver = webdriver.Chrome()

In [48]:
driver.get(url)


In [49]:
table = driver.find_element_by_xpath("/html/body/div[3]/div[3]/div/div[6]/div[3]/div/table")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=101.0.4951.54)


In [7]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [8]:
odd_class = soup.find_all('tr', class_='odd')

In [22]:
odd_class[2].find_all('td')

[<td style="font-size:12px;color: grey;text-align:center;vertical-align:middle;">1</td>,
 <td style="font-weight: bold; font-size:15px; text-align:left;"><a class="mt_a" href="country/us/">USA</a></td>,
 <td class="sorting_1" style="font-weight: bold; text-align:right">83,953,371</td>,
 <td style="font-weight: bold; text-align:right;"></td>,
 <td style="font-weight: bold; text-align:right;">1,025,764 </td>,
 <td style="font-weight: bold; 
                                     text-align:right;"></td>,
 <td style="font-weight: bold; text-align:right">81,107,296</td>,
 <td style="font-weight: bold; text-align:right;"></td>,
 <td style="text-align:right;font-weight:bold;">1,820,311</td>,
 <td style="font-weight: bold; text-align:right">1,794</td>,
 <td style="font-weight: bold; text-align:right">250,905</td>,
 <td style="font-weight: bold; text-align:right">3,066</td>,
 <td style="font-weight: bold; text-align:right">1,014,741,268</td>,
 <td style="font-weight: bold; text-align:right">3,03

In [25]:
index = []
country = []
total_cases = []
new_cases = []
total_deaths = []
new_deaths = []
total_recovered = []
new_recovered = []
active_cases = []
serious = []
tot_cases_per_pop = []
death_per_pop = []
total_test = []
tests_per_pop = []
population = []
region = []

In [26]:
for i in odd_class:
    att = i.find_all('td')
    if len(att)!=16:
        continue
    else:
        if att[0].text == '' or att[1].text == '':
            continue
        else:
            for j in range(16):
                if (j==0):
                    index.append(att[j].text)
                elif(j==1):
                    country.append(att[j].text)
                elif(j==2):
                    total_cases.append(att[j].text)
                elif(j==3):
                    new_cases.append(att[j].text)
                elif(j==4):
                    total_deaths.append(att[j].text)
                elif(j==5):
                    new_deaths.append(att[j].text)
                elif(j==6):
                    total_recovered.append(att[j].text)
                elif(j==7):
                    new_recovered.append(att[j].text)
                elif(j==8):
                    active_cases.append(att[j].text)
                elif(j==9):
                    serious.append(att[j].text)
                elif(j==10):
                    tot_cases_per_pop.append(att[j].text)
                elif(j==11):
                    death_per_pop.append(att[j].text)
                elif(j==12):
                    total_test.append(att[j].text)
                elif(j==13):
                    tests_per_pop.append(att[j].text)
                elif(j == 14):
                    population.append(att[j].text)
                elif(j == 15):
                    region.append(att[j].text)

In [27]:
data ={'Country': country, 'Total Cases': total_cases , 'New Cases': new_cases,
      "Total Deaths": total_deaths, "New Deaths": new_deaths, "Total Recovered": total_recovered,
      "New Recovered": new_recovered, "Active Cases": active_cases, "Serious": serious,
      "Total Cases/1M pop": tot_cases_per_pop, "Death/1M pop": death_per_pop,"Total tests":total_test,
      "Tests/ 1M pop": tests_per_pop, "Population":population, "Region": region}

In [28]:
df = pd.DataFrame(data)

In [29]:
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious,Total Cases/1M pop,Death/1M pop,Total tests,Tests/ 1M pop,Population,Region
0,USA,"83,953,371",,"1,025,764",,"81,107,296",,"1,820,311","1,794","250,905","3,066","1,014,741,268","3,032,684","334,601,743",North America
1,India,"43,113,413",,"524,181",,"42,570,165",,"19,067",698,"30,682",373,"842,458,167","599,545","1,405,161,912",Asia
2,France,"29,061,523",,"147,075",,"27,928,942",,"985,506","1,364","443,408","2,244","269,816,888","4,116,745","65,541,322",Europe
3,UK,"22,145,157",,"176,424",,"21,616,575",,"352,158",253,"323,064","2,574","519,264,096","7,575,273","68,547,249",Europe
4,S. Korea,"17,694,677","+35,883","23,554",+63,N/A,N/A,N/A,354,"344,583",459,"15,804,065","307,766","51,350,995",Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,Falkland Islands,197,,,,N/A,N/A,N/A,,"53,708",,"8,632","2,353,326","3,668",South America
338,Vatican City,29,,,,29,,0,,"36,025",,,,805,Europe
339,Western Sahara,10,,1,,9,,0,,16,2,,,"624,560",Africa
340,Niue,9,,,,8,,1,,"5,468",,,,"1,646",Australia/Oceania


In [36]:
filename = str(datetime.now())[0:11]+"_covid19"+".csv"

In [37]:
df.to_csv(filename)